In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import fiona

In [2]:
fiona.supported_drivers['KML'] = 'rw'

In [3]:
gdf = gpd.read_file('./Parkings.geojson')

In [4]:
gdf

,rd_seg_id,marker_id,bay_id,meter_id,rd_seg_dsc,last_edit,geometry
0,20194,4497W,9590,572B,King Street between Lonsdale Street and Little...,20201208140823,"MULTIPOLYGON (((144.95463 -37.81425, 144.95461..."
1,20194,4457W,9585,572B,King Street between Lonsdale Street and Little...,20201208140823,"MULTIPOLYGON (((144.95450 -37.81396, 144.95447..."
2,20194,4461W,9583,572B,King Street between Lonsdale Street and Little...,20201208140822,"MULTIPOLYGON (((144.95445 -37.81385, 144.95443..."
3,20194,4499W,9589,572B,King Street between Lonsdale Street and Little...,20201208140823,"MULTIPOLYGON (((144.95461 -37.81420, 144.95458..."
4,20194,4453W,9587,572B,King Street between Lonsdale Street and Little...,20201208140823,"MULTIPOLYGON (((144.95456 -37.81409, 144.95453..."
...,...,...,...,...,...,...,...
24710,21557,12803S,10289,None,Pelham Street between Drummond Street and Lygo...,20201208140902,"MULTIPOLYGON (((144.96697 -37.80295, 144.96697..."
24711,21341,None,4832,None,Abbotsford Street between Miller Street and Vi...,20201223122247,"MULTIPOLYGON (((144.94521 -37.80512, 144.94520..."
24712,21987,15056E,15215,None,Charles Street between Jolimont Street and Pal...,20201223122253,"MULTIPOLYGON (((144.98121 -37.81770, 144.98120..."
24713,21739,None,6977,None,Altona Street between Ormond Street and Kensin...,20201223122251,"MULTIPOLYGON (((144.92470 -37.79617, 144.92472..."


In [5]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [6]:
gdf['geometry'] = gdf['geometry'].to_crs(crs=4326)

gdf['geometry'] = gdf['geometry'].centroid

<ipython-input-6-99f9c083bc83>:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['geometry'] = gdf['geometry'].centroid


In [7]:
for index, value in gdf.iterrows():
    gdf.loc[index, 'x_coordinate'] = gdf.loc[index, 'geometry'].x
    gdf.loc[index, 'y_coordinate'] = gdf.loc[index, 'geometry'].y

In [8]:
gdf.drop('last_edit', inplace=True, axis=1)
gdf.drop('geometry', inplace=True, axis=1)

In [9]:
gdf.to_csv('./mb_parkings.csv')

In [10]:
df1 = pd.read_csv('./mb_parkings.csv')
df2 = pd.read_csv('./Parking Restrictions.csv')

In [11]:
df1.drop('Unnamed: 0', inplace=True, axis=1)

In [12]:
df1 = df1.rename(columns={"rd_seg_id": "RoadSegID", "marker_id": "MarkerID", "bay_id": "BayID", "meter_id": "MeterID",\
                   "rd_seg_dsc": "RoadSegDesc", "x_coordinate": "X_Coordinate", "y_coordinate": "Y_Coordinate"})

In [13]:
df1

,RoadSegID,MarkerID,BayID,MeterID,RoadSegDesc,X_Coordinate,Y_Coordinate
0,20194.0,4497W,9590,572B,King Street between Lonsdale Street and Little...,144.954608,-37.814225
1,20194.0,4457W,9585,572B,King Street between Lonsdale Street and Little...,144.954475,-37.813938
2,20194.0,4461W,9583,572B,King Street between Lonsdale Street and Little...,144.954425,-37.813831
3,20194.0,4499W,9589,572B,King Street between Lonsdale Street and Little...,144.954583,-37.814171
4,20194.0,4453W,9587,572B,King Street between Lonsdale Street and Little...,144.954533,-37.814063
...,...,...,...,...,...,...,...
24710,21557.0,12803S,10289,NaN,Pelham Street between Drummond Street and Lygo...,144.966931,-37.802955
24711,21341.0,NaN,4832,NaN,Abbotsford Street between Miller Street and Vi...,144.945191,-37.805145
24712,21987.0,15056E,15215,NaN,Charles Street between Jolimont Street and Pal...,144.981193,-37.817718
24713,21739.0,NaN,6977,NaN,Altona Street between Ormond Street and Kensin...,144.924688,-37.796192


In [14]:
df1['BayID'].isnull().sum()

0

In [15]:
df2['BayID'].isnull().sum()

0

In [16]:
df2.dropna(how='all')

,BayID,DeviceID,Description1,Description2,Description3,Description4,Description5,Description6,DisabilityExt1,DisabilityExt2,...,ToDay3,ToDay4,ToDay5,ToDay6,TypeDesc1,TypeDesc2,TypeDesc3,TypeDesc4,TypeDesc5,TypeDesc6
0,8634,28003,2P RPA 1B 7:30-23:00 M-SUN,NaN,NaN,NaN,NaN,NaN,240,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN
1,8587,27956,2P RPA 1B 7:30-23:00 M-SUN,NaN,NaN,NaN,NaN,NaN,240,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN
2,8498,27876,2P RPA 1B 7:30-23:00 M-SUN,NaN,NaN,NaN,NaN,NaN,240,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN
3,3156,26011,3P MTR M-SAT 7:30-20:30,2P SUN 7:30-18:30,NaN,NaN,NaN,NaN,360,240.0,...,NaN,NaN,NaN,NaN,3P Meter,2P,NaN,NaN,NaN,NaN
4,8167,25546,1P M-SUN 7:30-23:00,NaN,NaN,NaN,NaN,NaN,120,NaN,...,NaN,NaN,NaN,NaN,1P,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4223,3134,24600,3P MTR M-SAT 7:30-20:30,2P SUN 7:30-18:30,NaN,NaN,NaN,NaN,360,240.0,...,NaN,NaN,NaN,NaN,3P Meter,2P,NaN,NaN,NaN,NaN
4224,6076,30186,LZ 15MINS 7.30am - 7.30pm,NaN,NaN,NaN,NaN,NaN,0,NaN,...,NaN,NaN,NaN,NaN,Loading Zone 15M,NaN,NaN,NaN,NaN,NaN
4225,3136,30610,3P MTR M-SAT 7:30-20:30,2P SUN 7:30-18:30,NaN,NaN,NaN,NaN,360,240.0,...,NaN,NaN,NaN,NaN,3P Meter,2P,NaN,NaN,NaN,NaN
4226,8182,30142,1P M-SUN 7:30-23:00,NaN,NaN,NaN,NaN,NaN,120,NaN,...,NaN,NaN,NaN,NaN,1P,NaN,NaN,NaN,NaN,NaN


In [17]:
df = pd.merge(df1, df2, left_on = 'BayID', right_on = 'BayID', how='right')

In [18]:
df

,RoadSegID,MarkerID,BayID,MeterID,RoadSegDesc,X_Coordinate,Y_Coordinate,DeviceID,Description1,Description2,...,ToDay3,ToDay4,ToDay5,ToDay6,TypeDesc1,TypeDesc2,TypeDesc3,TypeDesc4,TypeDesc5,TypeDesc6
0,20068.0,1503W,8634,NaN,Intersection of Spencer Street and Flinders Lane,144.954535,-37.820167,28003,2P RPA 1B 7:30-23:00 M-SUN,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN
1,20011.0,C3118,8587,NaN,Lonsdale Street between Queen Street and Willi...,144.958118,-37.813491,27956,2P RPA 1B 7:30-23:00 M-SUN,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN
2,21848.0,C80,8498,NaN,Spring Street between Lonsdale Street and Litt...,144.971952,-37.809023,27876,2P RPA 1B 7:30-23:00 M-SUN,NaN,...,NaN,NaN,NaN,NaN,2P,NaN,NaN,NaN,NaN,NaN
3,21461.0,NaN,3156,NaN,Hawke Street between Ireland Street and Spence...,144.946068,-37.807778,26011,3P MTR M-SAT 7:30-20:30,2P SUN 7:30-18:30,...,NaN,NaN,NaN,NaN,3P Meter,2P,NaN,NaN,NaN,NaN
4,21811.0,NaN,8167,NaN,Cakebread Mews between Stockmans Way and Gateh...,144.918994,-37.793029,25546,1P M-SUN 7:30-23:00,NaN,...,NaN,NaN,NaN,NaN,1P,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4246,21228.0,NaN,3134,NaN,Dryburgh Street between Canning Street and Ers...,144.944744,-37.795295,24600,3P MTR M-SAT 7:30-20:30,2P SUN 7:30-18:30,...,NaN,NaN,NaN,NaN,3P Meter,2P,NaN,NaN,NaN,NaN
4247,21325.0,NaN,6076,NaN,Green Street between Fogarty Street and Langfo...,144.938848,-37.798609,30186,LZ 15MINS 7.30am - 7.30pm,NaN,...,NaN,NaN,NaN,NaN,Loading Zone 15M,NaN,NaN,NaN,NaN,NaN
4248,21844.0,NaN,3136,NaN,Kensington Road between Altona Street and Merc...,144.923112,-37.796831,30610,3P MTR M-SAT 7:30-20:30,2P SUN 7:30-18:30,...,NaN,NaN,NaN,NaN,3P Meter,2P,NaN,NaN,NaN,NaN
4249,NaN,NaN,8182,NaN,NaN,NaN,NaN,30142,1P M-SUN 7:30-23:00,NaN,...,NaN,NaN,NaN,NaN,1P,NaN,NaN,NaN,NaN,NaN


In [19]:
df['MarkerID'] = df['MarkerID'].fillna('Marker ID unknown')
df['MeterID'] = df['MeterID'].fillna('Meter ID unknown')
df['RoadSegID'] = df['RoadSegID'].fillna('Road Seg ID unknown')
df['RoadSegDesc'] = df['RoadSegDesc'].fillna('Road Seg Desc unknown')

In [20]:
for index in df['TypeDesc1'].notna().index:
    df = df[df['TypeDesc1'].str.contains('Disabled', regex=False)]

In [21]:
df

,RoadSegID,MarkerID,BayID,MeterID,RoadSegDesc,X_Coordinate,Y_Coordinate,DeviceID,Description1,Description2,...,ToDay3,ToDay4,ToDay5,ToDay6,TypeDesc1,TypeDesc2,TypeDesc3,TypeDesc4,TypeDesc5,TypeDesc6
83,21035.0,Marker ID unknown,8457,Meter ID unknown,Queensberry Street between Lothian Street and ...,144.943977,-37.802522,27818,3P DIS ONLY M-SUN 7:30am-6:30pm,NaN,...,NaN,NaN,NaN,NaN,3P Disabled Only,NaN,NaN,NaN,NaN,NaN
100,20884.0,Marker ID unknown,595,Meter ID unknown,Nicholson Street between Kay Street and Prince...,144.975376,-37.794336,27454,P DIS ONLY 20.30 - 23.59,2P DIS 7.30 - 8.30PM,...,NaN,NaN,NaN,NaN,Disabled Only,2P Disabled Only,NaN,NaN,NaN,NaN
263,21035.0,Marker ID unknown,8456,Meter ID unknown,Queensberry Street between Lothian Street and ...,144.944048,-37.802530,27817,3P DIS ONLY M-SUN 7:30am-6:30pm,NaN,...,NaN,NaN,NaN,NaN,3P Disabled Only,NaN,NaN,NaN,NaN,NaN
301,22520.0,Marker ID unknown,6615,Meter ID unknown,Bent Street between Chelmsford Street and Hard...,144.935152,-37.796095,30265,2P Disabled Only,NaN,...,NaN,NaN,NaN,NaN,2P Disabled Only,NaN,NaN,NaN,NaN,NaN
317,20127.0,Marker ID unknown,6081,Meter ID unknown,Flinders Lane between Swanston Street and Eliz...,144.965226,-37.817009,30248,2P DIS M-SAT 7:30-18:30,NaN,...,NaN,NaN,NaN,NaN,2P Disabled Only,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4126,20889.0,12247S,6173,VS5,Albert Street between Nicholson Street and Gis...,144.975072,-37.809575,30249,2P DIS M-SAT 7:30-18:30,NaN,...,NaN,NaN,NaN,NaN,2P Disabled Only,NaN,NaN,NaN,NaN,NaN
4148,22473.0,Marker ID unknown,6237,Meter ID unknown,Slater Street from St Kilda Road,144.977682,-37.838782,30306,2P DIS M-SUN 0:00-23:59,NaN,...,NaN,NaN,NaN,NaN,2P Disabled Only,NaN,NaN,NaN,NaN,NaN
4197,22408.0,Marker ID unknown,6466,Meter ID unknown,Southgate Street between Flemington Road and M...,144.943164,-37.788535,30286,2P DIS ONLY M-SUN 7:30-18:30,NaN,...,NaN,NaN,NaN,NaN,2P Disabled Only,NaN,NaN,NaN,NaN,NaN
4198,20889.0,12251S,6174,VS5,Albert Street between Nicholson Street and Gis...,144.975205,-37.809589,30192,2P DIS M-SAT 7:30-18:30,NaN,...,NaN,NaN,NaN,NaN,2P Disabled Only,NaN,NaN,NaN,NaN,NaN


In [22]:
df.drop('MarkerID', inplace=True, axis=1)
df.drop('MeterID', inplace=True, axis=1)

In [23]:
df = df.dropna(axis=1, how='all')

In [24]:
df = df[['BayID', 'RoadSegDesc', 'X_Coordinate', 'Y_Coordinate', 'Description1', 'Description2', 'Description3', 'Description4']]

In [25]:
df = df.fillna(' ')

In [26]:
df['Description'] = df['Description1'].str.cat(df['Description2'], sep='\n')
df['Description'] = df['Description'].str.cat(df['Description3'], sep='\n')
df['Description'] = df['Description'].str.cat(df['Description4'], sep='\n')

In [27]:
df.drop('Description1', inplace=True, axis=1)
df.drop('Description2', inplace=True, axis=1)
df.drop('Description3', inplace=True, axis=1)
df.drop('Description4', inplace=True, axis=1)

In [28]:
df.reset_index(drop=True, inplace=True)

In [29]:
df

,BayID,RoadSegDesc,X_Coordinate,Y_Coordinate,Description
0,8457,Queensberry Street between Lothian Street and ...,144.943977,-37.802522,3P DIS ONLY M-SUN 7:30am-6:30pm\n \n \n
1,595,Nicholson Street between Kay Street and Prince...,144.975376,-37.794336,P DIS ONLY 20.30 - 23.59\n2P DIS 7.30 - 8.30PM...
2,8456,Queensberry Street between Lothian Street and ...,144.944048,-37.802530,3P DIS ONLY M-SUN 7:30am-6:30pm\n \n \n
3,6615,Bent Street between Chelmsford Street and Hard...,144.935152,-37.796095,2P Disabled Only\n \n \n
4,6081,Flinders Lane between Swanston Street and Eliz...,144.965226,-37.817009,2P DIS M-SAT 7:30-18:30\n \n \n
...,...,...,...,...,...
93,6173,Albert Street between Nicholson Street and Gis...,144.975072,-37.809575,2P DIS M-SAT 7:30-18:30\n \n \n
94,6237,Slater Street from St Kilda Road,144.977682,-37.838782,2P DIS M-SUN 0:00-23:59\n \n \n
95,6466,Southgate Street between Flemington Road and M...,144.943164,-37.788535,2P DIS ONLY M-SUN 7:30-18:30\n \n \n
96,6174,Albert Street between Nicholson Street and Gis...,144.975205,-37.809589,2P DIS M-SAT 7:30-18:30\n \n \n


In [30]:
df.to_csv('./mb_parkings.csv')

In [33]:
df[df['BayID']==6086]

,BayID,RoadSegDesc,X_Coordinate,Y_Coordinate,Description
60,6086,Wellington Crescent from Wellington Parade South,144.978029,-37.817272,P DIS M-Sat 6.30-11.59\nP DIS Sun 00:00 - 11:5...
